# Quantum Chemistry: Dipole Calculation

In [ ]:
# Downgrade NumPy to version compatible with RDKit and PySCF
!pip install numpy==1.23.5

# Restart the runtime to apply the change before any imports
import os
os.kill(os.getpid(), 9)

In [ ]:
# Install PySCF and rdkit
!pip install pyscf 
!pip install rdkit-pypi 
!pip install pandas 
!pip install py3Dmol

In [ ]:
# Load Libraries
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto, scf
import pandas as pd
import numpy as np
import py3Dmol

def show_molecule_3D(rdkit_mol, size=(300, 300)):
    # Generate 3D coordinates
    mol_block = Chem.MolToMolBlock(rdkit_mol)
    
    # Create viewer
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mol_block, 'mol')
    viewer.setStyle({'stick': {}})
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    return viewer.show()

**PySCF:** Python-based Simulations of Chemistry Framework (PySCF) is an ab initio computational chemistry program natively implemented in Python program language.

**RDKit** is an open-source cheminformatics toolkit implemented in Python and C++. It provides robust tools for molecular representation, structure manipulation, and property calculation, enabling efficient handling of chemical information and integration into Python-based workflows.

**NumPy** is a fundamental package for scientific computing in Python. It provides efficient support for large, multi-dimensional arrays and matrices, along with a comprehensive collection of mathematical functions to perform element-wise operations, linear algebra, Fourier analysis, and more, forming the core of many numerical workflows in computational science.

**pandas** is a powerful, open-source data analysis and manipulation library for Python. It provides high-performance data structures like DataFrame and Series, enabling intuitive handling, transformation, and analysis of structured data within scientific and statistical workflows.

**py3Dmol** is a Python interface to the 3Dmol.js molecular visualization library. It enables interactive, high-quality 3D visualization of molecular structures directly within Jupyter and Colab notebooks, making it ideal for displaying geometries and computational chemistry results in a web-based environment.

In [ ]:
# Define molecules
molecules = {
    "water": "O",                      # Single oxygen atom → implies H2O with implicit hydrogens
    "ammonia": "N",                    # Single nitrogen atom → NH3 with implicit hydrogens
    "hydrogen_fluoride": "[H]F",       # Explicit hydrogen bonded to fluorine → HF
    "carbon_monoxide": "[C-]#[O+]",    # Formal negative charge on carbon, triple bond to positively charged oxygen
    "methane": "C"                     # Single carbon atom → CH4 with implicit hydrogens
}

**SMILES** (Simplified Molecular Input Line Entry System) is a compact, human-readable notation for describing molecular structures using ASCII strings. It encodes atoms, bonds, ring closures, charges, and stereochemistry in a linear form, allowing molecules to be easily stored, shared, and interpreted by cheminformatics tools such as RDKit.

In [ ]:
# Example: show ammonia
mol_rdkit = Chem.AddHs(Chem.MolFromSmiles("N"))
AllChem.EmbedMolecule(mol_rdkit, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol_rdkit)

show_molecule_3D(mol_rdkit)

In [ ]:
results = []

for name, smiles in molecules.items():
    # Generate 3D geometry
    mol_rdkit = Chem.AddHs(Chem.MolFromSmiles(smiles))
    AllChem.EmbedMolecule(mol_rdkit, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol_rdkit)
    conf = mol_rdkit.GetConformer()

    # Convert to PySCF format
    atoms = []
    for atom in mol_rdkit.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        atoms.append(f"{atom.GetSymbol()} {pos.x:.6f} {pos.y:.6f} {pos.z:.6f}")

    mol = gto.Mole()
    mol.atom = "\n".join(atoms)    # Coordinates from RDKit
    mol.basis = "def2svp"          # Double-zeta basis set with polarization
    mol.charge = 0                 # Neutral molecule
    mol.spin = 0                   # Closed-shell → RHF applies
    mol.build()
    
    mf = scf.RHF(mol)              # Restricted Hartree–Fock for closed-shell systems
    energy = mf.kernel()           # Compute SCF total energy

    # Dipole
    dip = mf.dip_moment()          # Compute dipole moment (Debye) and returns (dx, dy, dz)
    dip_magnitude = np.linalg.norm(dip)

    # Store results
    results.append({
        "Molecule": name,
        "HF Energy (Hartree)": energy,
        "Dipole Moment (D)": f"({dip[0]:.3f}, {dip[1]:.3f}, {dip[2]:.3f})",
        "Dipole |μ| (D)": dip_magnitude
    })

# Display as DataFrame
df = pd.DataFrame(results)
df

**Hartree–Fock (HF)** is a fundamental quantum chemistry method that approximates the electronic structure of molecules using a self-consistent field approach. It represents electrons as independent particles in a mean-field generated by the others, producing molecular orbitals and total electronic energy. In its restricted form (RHF), all electrons are paired in spatial orbitals, making it suitable for closed-shell systems.

**Basis sets** are predefined collections of mathematical functions (typically Gaussians) used to represent atomic orbitals in quantum chemistry calculations. The **def2-SVP** basis set is a split-valence double-zeta basis from the def2 family, balancing computational efficiency and accuracy for elements up to radon. It includes polarization functions for better angular flexibility in bonding environments.